In [1]:
import sys
sys.path.append('..')

In [2]:
import os
import csv

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset
import pandas as pd
from pandas.api.types import CategoricalDtype

In [3]:
from defences.util import score
from models.numeric import NumericModel
from experiments.train_pt import predict

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
PATH_OUTPUTS = os.path.join('..', 'csv')
if not os.path.exists(PATH_OUTPUTS):
    os.makedirs(PATH_OUTPUTS)

In [6]:
def predict(model, X, device, batch_size=512):
    dataset = TensorDataset(torch.from_numpy(X))
    loader = DataLoader(dataset, batch_size, shuffle=False)
    model.eval()
    batch = next(iter(loader))
    y = model(batch[0].to(device))
    shape_output = (len(loader.dataset), y.size(1))
    outputs = torch.zeros(shape_output, dtype=torch.float32)

    start = 0
    with torch.no_grad():
        for batch in loader:
            x = batch[0].to(device)
            end = start + x.size(0)
            outputs[start:end] = model(x)
            start = end

    return outputs.max(1)[1].type(torch.long).cpu().detach().numpy()

In [7]:
DATA = {
    'mnist': {'n_features': (1, 28, 28), 'n_classes': 10},
    'cifar10': {'n_features': (3, 32, 32), 'n_classes': 10},
    'banknote': {'file_name': 'banknote_preprocessed.csv', 'n_features': 4, 'n_test': 400, 'n_classes': 2},
    'htru2': {'file_name': 'htru2_preprocessed.csv', 'n_features': 8, 'n_test': 4000, 'n_classes': 2},
    'segment': {'file_name': 'segment_preprocessed.csv', 'n_features': 18, 'n_test': 400, 'n_classes': 7},
    'texture': {'file_name': 'texture_preprocessed.csv', 'n_features': 40, 'n_test': 600, 'n_classes': 11}}

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
def save_results(data_name, path_model, model_prefix, attacks_name, defences_name):
    n_features = DATA[data_name]['n_features']
    n_classes = DATA[data_name]['n_classes']

    model = NumericModel(
        n_features,
        n_hidden=n_features * 4,
        n_classes=n_classes,
        use_prob=True).to(device)
    model.load_state_dict(torch.load(path_model))

    path_file = os.path.join(PATH_OUTPUTS, '{}.csv'.format(model_prefix))
    print('Save to:', path_file)
    with open(path_file, mode='w') as file:
        file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        title = ['Attack', 'No defence', 'Defence', 'Adv only','Mixed', 'FPR']
        print(', '.join(title))
        file_writer.writerow(title)

        for attack in attacks_name:
            for defence in defences_name:
                file_name = '{}_{}_{}.pt'.format(model_prefix, attack, defence)
                path_data = os.path.join('..', 'results', file_name)
                data = torch.load(path_data)
                # print(file_name)

                X_test = data['X_test']
                y_test = data['y_test']
                labels_test = data['labels_test']
                res_test = data['res_test']

                idx_adv = np.where(labels_test==1)[0]
                idx_benign = np.where(labels_test==0)[0]

                n_adv = np.sum(labels_test==1)
                n_benign = np.sum(labels_test==0)

                pred_test = predict(model, X_test, device=device, batch_size=512)

                acc_no_defence = np.sum(pred_test[idx_adv] == y_test[idx_adv]) / n_adv
                acc_defence = score(
                    res_test[idx_adv],
                    y_test[idx_adv],
                    pred_test[idx_adv],
                    labels_test[idx_adv])
                acc_mixed = score(res_test, y_test, pred_test, labels_test)

                fp = np.sum(res_test[idx_benign] == 1)
                fpr = fp / len(X_test)

                row_value = [
                    attack,
                    np.round(acc_no_defence*100, 2),
                    defence,
                    np.round(acc_defence*100, 2),
                    np.round(acc_mixed*100, 2),
                    np.round(fpr*100, 2)]
                print(', '.join([str(val) for val in row_value]))
                file_writer.writerow(row_value)

In [10]:
defences_image = ['baard_2stage','baard_3stage', 'fs', 'lid', 'magnet','rc']
defences_num = ['baard_2stage','baard_3stage', 'fs', 'lid', 'rc']

In [11]:
attacks = [
    'apgd_0.05',
    'apgd_0.2',
    'boundary_0.3',
    'cw2_0.01',
    'cw2_100.0',
    'deepfool_0.001',
    'deepfool_1e-06',
    'fgsm_0.05',
    'fgsm_0.2']

In [12]:
PATH_MODEL = os.path.join('..', 'results', 'banknote_400.pt')
save_results(
    data_name='banknote', 
    path_model=PATH_MODEL, 
    model_prefix='banknote_basic16', 
    attacks_name=attacks, 
    defences_name=defences_num)

Save to: ../csv/banknote_basic16.csv
Attack, No defence, Defence, Adv only, Mixed, FPR
apgd_0.05, 77.55, baard_2stage, 99.49, 88.01, 11.73
apgd_0.05, 77.55, baard_3stage, 99.49, 68.62, 31.12
apgd_0.05, 77.55, fs, 92.35, 63.27, 32.91
apgd_0.05, 77.55, lid, 91.33, 93.62, 2.04
apgd_0.05, 77.55, rc, 82.65, 67.86, 23.47
apgd_0.2, 0.51, baard_2stage, 96.94, 92.6, 5.87
apgd_0.2, 0.51, baard_3stage, 96.94, 88.27, 10.2
apgd_0.2, 0.51, fs, 40.82, 48.98, 21.43
apgd_0.2, 0.51, lid, 67.35, 82.4, 1.28
apgd_0.2, 0.51, rc, 52.04, 52.55, 23.47
boundary_0.3, 46.43, baard_2stage, 97.96, 91.58, 7.4
boundary_0.3, 46.43, baard_3stage, 100.0, 90.05, 9.95
boundary_0.3, 46.43, fs, 50.0, 66.33, 8.67
boundary_0.3, 46.43, lid, 76.02, 86.22, 1.79
boundary_0.3, 46.43, rc, 61.73, 57.4, 23.47
cw2_0.01, 0.0, baard_2stage, 97.45, 92.6, 6.12
cw2_0.01, 0.0, baard_3stage, 100.0, 91.33, 8.67
cw2_0.01, 0.0, fs, 64.29, 53.83, 28.32
cw2_0.01, 0.0, lid, 74.49, 83.67, 3.57
cw2_0.01, 0.0, rc, 32.14, 42.6, 23.47
cw2_100.0, 0.0, b